# Homework 3: Language Modelling in Hangman

Student Name: Yu Dong

Student ID: 928922

## General info

<b>Due date</b>:  Friday, 17 May 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models. Your objective is to create an automatic player which makes the fewest mistakes.

## The Hangman Game (*No implementation is needed*)

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. 

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided gusser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('Enter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to True. When submitting your solution, set to False so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = True

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
a
Guess is a
Good guess: _ _ a _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
e
Guess is e
Good guess: _ _ a _ e _ e _
You have 8 attempts remaining.
Enter your guess:
c
Guess is c
Sorry, try again.
You have 7 attempts remaining.
Enter your guess:
v
Guess is v
Good guess: _ _ a _ e v e _
You have 7 attempts remaining.
Enter your guess:
r
Guess is r
Good guess: _ _ a _ e v e r
You have 7 attempts remaining.
Enter your guess:
w
Guess is w
Good guess: w _ a _ e v e r
You have 7 attempts remaining.
Enter your guess:
h
Guess is h
Good guess: w h a _ e v e r
You have 7 attempts remaining.
Enter your guess:
t
Guess is t
Good guess: w h a t e v e r
Congratulations, you won.


## 1. Preparing Test Set and Training Set (1 mark)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the algorithm. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown` and the `words` method, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(1 mark)

In [5]:
from nltk.corpus import brown
import numpy as np
import re

np.random.seed(12345)

# word_set stores all the unique word types in the Brown corpus
word_set = []
# test_set stores 1000 word types for testing
test_set = []
# training_set stores the rest word types for training
training_set = []

###
# Your answer BEGINS HERE
###

#process paragraph in brown, use a set to store the words in each paragraph.
wordset = set()
#remove the character which cannot match a-zA-Z and lowercase the word
for word in brown.words():
    if re.match(r'^[a-zA-Z]+$',word):
        if word.lower() not in wordset:
            wordset.add(word.lower())
#split test set and training set
word_set = np.asarray(list(wordset))
np.random.shuffle(word_set)
test_set = word_set[:1000]
training_set = word_set[1000:]

###
# Your answer ENDS HERE
###

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


<b>For your testing:</b>

In [6]:
assert(len(word_set) > 35000 and len(word_set) < 45000)

In [7]:
assert(len(test_set) == 1000)

In [8]:
assert(len(training_set) + len(test_set) == len(word_set))

In [9]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ length 7
You have 8 attempts remaining.
Enter your guess:
a
Guess is a
Sorry, try again.
You have 7 attempts remaining.
Enter your guess:
c
Guess is c
Sorry, try again.
You have 6 attempts remaining.
Enter your guess:
e
Guess is e
Good guess: _ e _ _ _ _ _
You have 6 attempts remaining.
Enter your guess:
i
Guess is i
Good guess: _ e _ _ i _ _
You have 6 attempts remaining.
Enter your guess:
q
Guess is q
Sorry, try again.
You have 5 attempts remaining.
Enter your guess:
y
Guess is y
Sorry, try again.
You have 4 attempts remaining.
Enter your guess:
n
Guess is n
Good guess: _ e _ _ i n _
You have 4 attempts remaining.
Enter your guess:
l
Guess is l
Good guess: _ e l _ i n _
You have 4 attempts remaining.
Enter your guess:
g
Guess is g
Good guess: _ e l _ i n g
You have 4 attempts remaining.
Enter your guess:
b
Guess is b
Sorry, try again.
You have 3 attempts remaining.
Enter your guess:
d
Guess is d
Sorry, try again.
You have 2 attempts rema

## 2. Simple Guesser: Random Guessing (1 mark)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

(1 mark)

In [10]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [11]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        This function implements a random guesser. It returns the random guess. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    #create a alphabetic characters list
    alpha_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    #create a list to store guessed characters
    store_guessed = [i for i in guessed]
    #create a list to store the character that can be selected
    choice_list = []
    #select characeters from alphabetic list which is not in guessed list
    for i in alpha_list:
        if i not in store_guessed:
            choice_list.append(i)
    return np.random.choice(choice_list)

    ###
    # Your answer ENDS HERE
    ###

# uncomment to run a single hangman game with output shown (useful for debugging)
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  16.702


<b>For your testing:</b>

In [12]:
assert(result > 10 and result < 20)

## 3. Your First AI Guesser: Unigram Guesser (1 mark)

**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

Hint: It should be much lower than random guessing.

(1 mark)

In [13]:
from collections import Counter

# unigram_counts stores the frequencies of characters over all training words
unigram_counts = Counter()

###
# Your answer BEGINS HERE
###

for word in training_set:
    for character in word:
        unigram_counts[character] += 1
        
###
# Your answer ENDS HERE
###

print(unigram_counts)

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        This function implements a unigram guesser. It returns a guess based on the unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    #create a order list to store the characeters with highest frequencies
    order_list = []
    #create a list to store guessed characters
    store_guessed = [i for i in guessed]
    #append the character to order list if it has highest frequency
    for (char,_) in unigram_counts.most_common():
        order_list.append(char)
    #select character from order list if it is not in guessed list
    for i in order_list:
        if i not in store_guessed:
            return i
        
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Counter({'e': 35188, 'i': 26057, 'a': 24369, 's': 23688, 'n': 22611, 'r': 22102, 't': 20667, 'o': 19184, 'l': 16667, 'c': 12618, 'd': 12276, 'u': 10206, 'm': 8704, 'g': 8610, 'p': 8514, 'h': 7339, 'b': 5730, 'y': 5367, 'f': 4223, 'v': 3439, 'k': 2979, 'w': 2815, 'z': 1035, 'x': 833, 'j': 637, 'q': 534})

Average number of incorrect guesses:  10.366


<b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

## 4. Your Second AI Guesser: Length-based Unigram Guesser (1 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having a *different* unigram model for each length of the words. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. You need to decide on how to handle this situation.

(1 mark)

In [15]:
from collections import defaultdict

# unigram_counts_by_length stores a dictionary, mapping word length to the frequencies of characters of words with that word length
unigram_counts_by_length = defaultdict(Counter)

###
# Your answer BEGINS HERE
###

for word in training_set:
    for character in word:
        unigram_counts_by_length[len(word)][character] += 1

###
# Your answer ENDS HERE
###

lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1
print(lengths)
print()
print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    """
        This function implements a length-based unigram guesser. It returns a guess based on the length-based unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    #create alphabetic set to store 26 characters
    alpha_set = {'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'}
    #if the length of the word not in lengths, make random choice from unguessed alphabetic set
    if len(mask) not in lengths:
        return np.random.choice(list(alpha_set-guessed))
    #create a list to store characters with highest frequencies for each length of word
    order_list = []
    for (char, _) in unigram_counts_by_length[len(mask)].most_common():
        order_list.append(char)
        for i in order_list:
            if i not in guessed:
                return i
    return np.random.choice(list(alpha_set-guessed))
    
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("Average number of incorrect guesses: ", result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

defaultdict(<class 'collections.Counter'>, {1: Counter({'b': 1, 'a': 1, 'x': 1, 'm': 1, 'o': 1, 't': 1, 'u': 1, 's': 1, 'g': 1, 'n': 1, 'p': 1, 'z': 1, 'y': 1, 'c': 1, 'e': 1, 'r': 1, 'w': 1, 'f': 1, 'k': 1, 'd': 1, 'h': 1, 'i': 1, 'l': 1, 'j': 1, 'q': 1, 'v': 1}), 2: Counter({'a': 24, 'e': 21, 'o': 19, 'm': 19, 's': 18, 'i': 16, 'l': 13, 'p': 12, 'h': 12, 't': 11, 'u': 11, 'n': 11, 'c': 10, 'f': 10, 'd': 10, 'r': 9, 'b': 8, 'w': 7, 'y': 6, 'g': 6, 'v': 6, 'k': 5, 'j': 5, 'x': 2, 'q': 2, 'z': 1}), 3: Counter({'a': 238, 'e': 198, 'o': 178, 'i': 129, 'n': 117, 's': 113, 't': 112, 'm': 103, 'u': 101, 'p': 101, 'd': 100, 'r': 99, 'b': 90, 'l': 79, 'g': 76, 'y': 75, 'w': 74, 'h': 74, 'c': 70, 'f': 51, 'k': 32, 'j': 28, 'v': 25, 'x': 22, 'z': 10, 'q': 6}), 4: Counter({'e': 894, 'a': 864, 's': 686, 'o': 676, 'l': 555, 'i': 533, 'r': 523, 't': 487, 'n': 466, 'd': 360, 'u': 350, 'm': 295, 'h': 290, 'p': 282, 'c': 2

<b>For your testing:</b>

In [16]:
assert(result > 5 and result < 15)

## 5. Your Third AI Guesser: Bigram Guesser (1 mark)

**Instructions:** Now for the next challenge, using a *bigram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

You should develop a *bigram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with a sentinel symbol `$`.) You should use *linear interpolation* to smooth between the higher order and lower order models, and you will have to decide how to weight each component (be reminded that all probabilities should sum to 1).

Your bigram guesser should apply your language model to each blank position in the secret word by using the left context as is known. E.g., in the partial word `$ _ e c _ e _ _` we know the left context for the first three blanks, but have no known left context for the last blank. Using a bigram language model, we are able to apply it to the first three blanks only. You should then select the character with the highest probability of predicting the most number of correct entries over the blanks. 

Do you see any improvement over the unigram guessers above?

(1 mark)

In [19]:
###
# Your answer BEGINS HERE
###

bigram_counts = defaultdict(Counter) # you will want a different data structure to store the bigram 
for word in training_set:
    word = "$" + word
    bigram_list = zip(word[:-1], word[1:])
    for bigram in bigram_list:
        first, second = bigram
        bigram_counts[first][second] += 1
#print(bigram_counts)
          
###
# Your answer ENDS HERE
###

import math
#input the character, previous characters, bigram_counts and the weight of bigram model
#return the probability of interpolation
def get_prob_interp(prev_char, char, bigram_counts, lambdas):
    total_count = float(sum(bigram_counts[prev_char].values()))
    interp_prob = 0.0
    if total_count != 0:
        interp_prob = bigram_counts[prev_char][char] / total_count
    return lambdas * interp_prob + (1 - lambdas) * unigram_counts[char] / float(sum(unigram_counts.values()))
def get_log_prob_interp(chars_list, char, bigram_counts, lambdas):
    log_prob = 0
    for _, chars in enumerate(chars_list):
        if len(chars) >= 2:
            chars = chars[-1:]
        log_prob += math.log(get_prob_interp(chars, char, bigram_counts, lambdas))
    return log_prob

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    """
        This function implements a bigram guesser. It returns a guess based on the bigram model using linear interpolation.
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    #use weighted bigram model to get the character with highest probability
    #the higher weight of bigram model, the lower average number of incorrect guesses
    mask = "$" + "".join(mask)
    chars_list = mask.split("_")[:-1]
    char_prob = dict()
    alpha_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    for i in alpha_list:
        if not i in guessed:
            char_prob[i] = get_log_prob_interp(chars_list, i, bigram_counts, 0.9)
    return max(char_prob, key=char_prob.get)
    
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), bigram_guesser, 10, True)

result = test_guesser(bigram_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  8.654


<b>For your testing:</b>

In [20]:
assert(result < 13)

## 6. Your Own AI Guesser (1 mark)

**Instructions:** You should try to develop a more effective AI, `my_ai_guesser`, for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, and using ngram models. Ensure you report the test performance of your guesser. Have fun!

You will be marked based on the explanation of your approach and its accuracy. 

(1 mark) 

In [33]:
###
# Your answer BEGINS HERE
###

trigram_counts = defaultdict(Counter) # you will want a different data structure to store the bigram 
for word in training_set:
    word = "$$" + word
    trigram_list = [word[i:i+3] for i,_ in enumerate(word[:-2])]
    for trigram in trigram_list:
        first, second = trigram[:-1], trigram[-1]
        trigram_counts[first][second] += 1
#print(trigram_counts)

def get_prob_interp(prev_char, char, trigram_counts, lambdas):
    total_count = float(sum(trigram_counts[prev_char].values()))
    interp_prob = 0.0
    if total_count!=0:
        interp_prob = trigram_counts[prev_char][char] / total_count
    return lambdas * interp_prob + (1 - lambdas) * unigram_counts[char] / float(sum(unigram_counts.values()))
def get_log_prob_interp(chars_list, char, trigram_counts, lambdas):
    log_prob = 0
    for index, chars in enumerate(chars_list):
        if len(chars) >= 3:
            chars = chars[-2:]
        log_prob += math.log(get_prob_interp(chars, char, trigram_counts, lambdas))
    return log_prob
def my_ai_guesser(mask, guessed,counts=trigram_counts):
    mask = "$$" + "".join(mask)
    chars_list = mask.split("_")[:-1]
    char_prob = dict()
    alpha_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    for i in alpha_list:
        if not i in guessed:
            char_prob[i] = get_log_prob_interp(chars_list, i, trigram_counts, 0.9)
    return max(char_prob, key=char_prob.get)

###
# Your answer ENDS HERE
###

result = test_guesser(my_ai_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  8.23


**Instructions:** Explain your approach and discuss your result below. Please keep it brief.

In [31]:
The above guesser is based on trigram model. The method weight the higher order model(trigram model) and lower order 
model (unigram model). The sum of weight probability is one. The guesser selects the character with highest probability.
The result of average number of incorrect guesses shows my_ai_guesser has better performance than previous guessers. 
Meanwhile, the higher weight of trigram model, the lower average number of incorrect guesses. Because trigram model 
selects character based on two previous characters, the unigram model selects character based on the frequency of 
character. The selection based on relationship between two previous characters will be more precise than the selection 
merely based on character frequency. 